In [1]:
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift
from collections import defaultdict
from datetime import datetime, timedelta, time
import sys
import folium


print("Python Version:", sys.version)
print("Pandas Version:", pd.__version__)
print("Numpy Version:", np.__version__)

main_path = '/Users/PC/Desktop/Insight/MTASafety'

Python Version: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
Pandas Version: 0.24.2
Numpy Version: 1.16.2


In [2]:
df = pd.read_csv(main_path + '/Predictions2.csv', low_memory=False)

In [3]:
def generateBaseMap(default_location=[40.74089545, -73.97906234], default_zoom_start=13):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [4]:
base_map = generateBaseMap()
base_map

In [5]:
from folium.plugins import HeatMap
base_map = generateBaseMap()
HeatMap(data=df[['Latitude2', 'Longitude2', 'Predictions']].groupby(['Latitude2', 'Longitude2']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

In [6]:
base_map.add_child(folium.ClickForMarker(popup='Potential Location'))

In [7]:
df['Numbers']=round(df['Predictions']*1000000)
df.shape

(20118, 103)

In [8]:
#Getting rid of 0s
df['Numbers']=(abs(df['Numbers']) + df['Numbers']) / 2

In [9]:
df2=df.reindex(df.index.repeat(df.Numbers))

In [10]:
df2.head()

,Unnamed: 0,X,Ident2,Complaint_x_x,entries_diff_x_x,exit_diff_x_x,OFNS_DESC_x_x,LAW_CAT_CD_x_x,Unnamed..0,Ident,...,timegroup_y,timegroupstr_y,wkdaynbr_y,Complaint_y,CombinedEntriesExits,RateofCrime,RateofCrimeJustEntry,weekend,Predictions,Numbers
0,1,0,1 AV_L01,2,73429,57190,"nan, nan, nan, nan, nan, nan, ASSAULT 3 & RELA...","nan, nan, nan, nan, nan, nan, MISDEMEANOR, MIS...",10,1 AV_L2017101,...,1.0,12am - 4am,0.0,1.0,130619,0.000015,0.000027,0,0.000014,14.0
0,1,0,1 AV_L01,2,73429,57190,"nan, nan, nan, nan, nan, nan, ASSAULT 3 & RELA...","nan, nan, nan, nan, nan, nan, MISDEMEANOR, MIS...",10,1 AV_L2017101,...,1.0,12am - 4am,0.0,1.0,130619,0.000015,0.000027,0,0.000014,14.0
0,1,0,1 AV_L01,2,73429,57190,"nan, nan, nan, nan, nan, nan, ASSAULT 3 & RELA...","nan, nan, nan, nan, nan, nan, MISDEMEANOR, MIS...",10,1 AV_L2017101,...,1.0,12am - 4am,0.0,1.0,130619,0.000015,0.000027,0,0.000014,14.0
0,1,0,1 AV_L01,2,73429,57190,"nan, nan, nan, nan, nan, nan, ASSAULT 3 & RELA...","nan, nan, nan, nan, nan, nan, MISDEMEANOR, MIS...",10,1 AV_L2017101,...,1.0,12am - 4am,0.0,1.0,130619,0.000015,0.000027,0,0.000014,14.0
0,1,0,1 AV_L01,2,73429,57190,"nan, nan, nan, nan, nan, nan, ASSAULT 3 & RELA...","nan, nan, nan, nan, nan, nan, MISDEMEANOR, MIS...",10,1 AV_L2017101,...,1.0,12am - 4am,0.0,1.0,130619,0.000015,0.000027,0,0.000014,14.0


In [11]:
df2['NumberAdd']=1

In [12]:
df_hour_list = []
for timegroup_x in df.timegroup_x.sort_values().unique():
    df_hour_list.append(df.loc[df.timegroup_x == timegroup_x, ['Latitude2', 'Longitude2']].groupby(['Latitude2', 'Longitude2']).sum().reset_index().values.tolist())

In [13]:
#Monday
base_mapMon=base_map
dfMon=df2[df2.weekday_x == 2].copy()
dfMon['NumberAdd']=1
dfMon_hour_list = []
for timegroup_x in dfMon.timegroup_x.sort_values().unique():
    dfMon_hour_list.append(dfMon.loc[dfMon.timegroup_x == timegroup_x, ['Latitude2', 'Longitude2', 'NumberAdd']].groupby(['Latitude2', 'Longitude2']).sum().reset_index().values.tolist())

In [14]:
df_times=[]
for timegroup_x in dfMon.timegroup_x.sort_values().unique():
    df_times.append(dfMon.loc[dfMon.timegroup_x == timegroup_x, ['timegroupstr_x']].groupby(['timegroupstr_x']).sum().reset_index().values.tolist())

In [15]:
from folium.plugins import HeatMapWithTime
base_mapMon = generateBaseMap(default_zoom_start=13)
HeatMapWithTime(dfMon_hour_list, index=df_times, radius=10,gradient={0: 'blue', .2: 'lime', .5: 'orange', .7: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_mapMon)
base_mapMon

In [16]:
#Saving Monday Map
base_mapMon.save(main_path+'Monday3.html')

In [17]:
#Tuesday
base_mapTues=base_map
dfTues=df2[df2.weekday_x == 3].copy()
dfTues['NumberAdd']=1
dfTues_hour_list = []
for timegroup_x in dfTues.timegroup_x.sort_values().unique():
    dfTues_hour_list.append(dfTues.loc[dfTues.timegroup_x == timegroup_x, ['Latitude2', 'Longitude2', 'NumberAdd']].groupby(['Latitude2', 'Longitude2']).sum().reset_index().values.tolist())

In [18]:
from folium.plugins import HeatMapWithTime
base_mapTues = generateBaseMap(default_zoom_start=13)
HeatMapWithTime(dfTues_hour_list, index=df_times, radius=10,gradient={0: 'blue', .2: 'lime', .5: 'orange', .7: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_mapTues)
base_mapTues

In [19]:
base_mapTues.save(main_path+'Tuesday3.html')

In [20]:
#Wednesday
base_mapWed=base_map
dfWed=df2[df2.weekday_x == 4].copy()
dfWed['NumberAdd']=1
dfWed_hour_list = []
for timegroup_x in dfWed.timegroup_x.sort_values().unique():
    dfWed_hour_list.append(dfWed.loc[dfWed.timegroup_x == timegroup_x, ['Latitude2', 'Longitude2', 'NumberAdd']].groupby(['Latitude2', 'Longitude2']).sum().reset_index().values.tolist())

In [21]:
from folium.plugins import HeatMapWithTime
base_mapWed = generateBaseMap(default_zoom_start=13)
HeatMapWithTime(dfWed_hour_list, index=df_times, radius=10,gradient={0: 'blue', .2: 'lime', .5: 'orange', .7: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_mapWed)
base_mapWed

In [22]:
base_mapWed.save(main_path+'Wednesday3.html')

In [23]:
#Thursday
base_mapThur=base_map
dfThur=df2[df2.weekday_x == 5].copy()
dfThur['NumberAdd']=1
dfThur_hour_list = []
for timegroup_x in dfThur.timegroup_x.sort_values().unique():
    dfThur_hour_list.append(dfThur.loc[dfThur.timegroup_x == timegroup_x, ['Latitude2', 'Longitude2', 'NumberAdd']].groupby(['Latitude2', 'Longitude2']).sum().reset_index().values.tolist())
from folium.plugins import HeatMapWithTime
base_mapThur = generateBaseMap(default_zoom_start=13)
HeatMapWithTime(dfThur_hour_list, index=df_times,radius=10,gradient={0: 'blue', .2: 'lime', .5: 'orange', .7: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_mapThur)
base_mapThur
base_mapThur.save(main_path+'Thursday3.html')

In [24]:
#Friday
base_mapFri=base_map
dfFri=df2[df2.weekday_x == 6].copy()
dfFri['NumberAdd']=1
dfFri_hour_list = []
for timegroup_x in dfFri.timegroup_x.sort_values().unique():
    dfFri_hour_list.append(dfFri.loc[dfFri.timegroup_x == timegroup_x, ['Latitude2', 'Longitude2', 'NumberAdd']].groupby(['Latitude2', 'Longitude2']).sum().reset_index().values.tolist())
from folium.plugins import HeatMapWithTime
base_mapFri = generateBaseMap(default_zoom_start=13)
HeatMapWithTime(dfFri_hour_list, index=df_times,radius=10,gradient={0: 'blue', .2: 'lime', .5: 'orange', .7: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_mapFri)
base_mapFri
base_mapFri.save(main_path+'Friday3.html')

In [25]:
#Saturday
base_mapSat=base_map
dfSat=df2[df2.weekday_x == 7].copy()
dfSat['NumberAdd']=1
dfSat_hour_list = []
for timegroup_x in dfSat.timegroup_x.sort_values().unique():
    dfSat_hour_list.append(dfSat.loc[dfSat.timegroup_x == timegroup_x, ['Latitude2', 'Longitude2', 'NumberAdd']].groupby(['Latitude2', 'Longitude2']).sum().reset_index().values.tolist())
from folium.plugins import HeatMapWithTime
base_mapSat = generateBaseMap(default_zoom_start=13)
HeatMapWithTime(dfSat_hour_list, index=df_times,radius=10,gradient={0: 'blue', .2: 'lime', .5: 'orange', .7: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_mapSat)
base_mapSat
base_mapSat.save(main_path+'Saturday3.html')

In [26]:
#Sunday
base_mapSun=base_map
dfSun=df2[df2.weekday_x == 1].copy()
dfSun['NumberAdd']=1
dfSun_hour_list = []
for timegroup_x in dfSun.timegroup_x.sort_values().unique():
    dfSun_hour_list.append(dfSun.loc[dfSun.timegroup_x == timegroup_x, ['Latitude2', 'Longitude2', 'NumberAdd']].groupby(['Latitude2', 'Longitude2']).sum().reset_index().values.tolist())
from folium.plugins import HeatMapWithTime
base_mapSun = generateBaseMap(default_zoom_start=13)
HeatMapWithTime(dfSun_hour_list,index=df_times,radius=10,gradient={0: 'blue', .2: 'lime', .5: 'orange', .7: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_mapSun)
base_mapSun
base_mapSun.save(main_path+'Sunday3.html')